# Move_Docutment(目前只能抓23年的資料)
雲端完成、Home完成、Company*未*完成
1..詢問關鍵字
2.生成目標路徑
3.複製目標資料夾，輸出路徑

In [ ]:
#簡化修正版
#cd C:\Users\OXO\OneDrive\01 Book\00 Test program
#pyinstaller -F Pack.py
import shutil
import os #用來避免檔案已經存在的異常
# TC_abbreviation = ["TPC", "TC(C0))", "CD", "23"]
# HT_abbreviation = ["HT", "D1", "CTC", "GEL", "23"]
Easy_Dict = {"1":"HT", "2":"TC", "99":""}
Plan_Dict = {"HT" : ["HT", "D1", "CTC", "GEL", "23"],
             "TC" : ["TPC", "TC(C0)", "CD", "23"]}
Cloud_HT = r"/content/sample_data/"
Cloud_TC = r"/content/sample_data/"
Home_HT = r"C:\Users\OXO\OneDrive\01 Book\00 Test program\HT"
Home_TC = r"C:\Users\OXO\OneDrive\01 Book\00 Test program\TC"
Source_Company_HT = r"\\10.162.10.58\全處共用區\_Dwg\興達電廠燃氣機組更新計畫"
destny_Company_HT = r"D:\00 興達計劃\05 EPC提供資料\HT"
Source_Company_TC = r"\\10.162.10.58\全處共用區\_Dwg\台中發電廠新建燃氣機組計畫"
destny_Company_TC = r"D:\00 台中計劃\05 EPC提供資料\TC"
Source_Customized = ""
destny_Customized = ""

#輸入 手動keyin
#輸出 計畫代號、完整文號
def Get_DocNo():
    # PlanNo :  1 = 興達計畫, 2 = 台中計畫, 99 = 指定路徑
    PlanNo = input("請輸入計畫\n興達請輸入\"1\" \n台中請輸入\"2\" \n自訂計畫請輸入\"99\":")
    DocNo = ""
    if PlanNo == "99" :
        DocNo = input("請輸入您的文號:")
    else:
        Doc_Num = str(input("請輸入號碼(1234):"))
        File_a = Plan_Dict.get(Easy_Dict.get(PlanNo, 'f'))
        DocNo = "-".join(File_a) + "-" + Doc_Num
    print("PlanNo:", PlanNo)
    print("DocNo:", DocNo)
    print("Get_DocNo() Done!")
    print("-------------------------------------------------------------")         
    return PlanNo, DocNo

#輸入 計畫代號、完整文號
#輸出 來源路徑、最終路徑
def Gen_path(PlanNo, DocNo):
    # PathWay : 1 = 雲端硬碟路徑  # 2 = 家庭硬碟路徑    # 3 = 公司硬碟路徑  # 0 = 指定路徑
    # PlanNo :  1 = 興達計畫, 2 = 台中計畫, 99 = 指定路徑
    path_map = {
        "11":(Cloud_HT, Cloud_HT),
        "12":(Cloud_TC, Cloud_TC),
        "21":(Home_HT, Home_HT),
        "22":(Home_TC, Home_TC),
        "31":(Source_Company_HT, destny_Company_HT),
        "32":(Source_Company_TC, destny_Company_TC),
        "0":("",""),
    }
    PathWay = input("請輸入使用路徑(0 = 自訂, 1 = 雲端, 2 = 家庭, 3 = 公司 ：)")
    source_folder = ""
    dest_folder = ""

    if PathWay == "0":
        source_folder = input("請輸入您的來源路徑 :") + "\\" + DocNo
        dest_folder = input("請輸入您的目的路徑 :") + "\\" + "new" +DocNo
    else:
        source_folder, dest_folder = path_map[PathWay + PlanNo][0] + "//" + DocNo, path_map[PathWay + PlanNo][1] + Easy_Dict.get(PlanNo, 'f') +"_" +DocNo
    print("source_folder:", source_folder)
    print("dest_folder:", dest_folder)
    print("Gen_path(PlanNo, DocNo), done!")
    print("-------------------------------------------------------------")   
    return source_folder, dest_folder
#輸入 來源路徑、最終路徑
#輸出 執行複製後，顯示最終路徑
def move_docutment(source_folder, dest_folder):
    #確認是否存在Source_folder
    if os.path.exists(source_folder):
        print("已找到指定資料夾")
        #若存在dest_folder，則刪除
        if os.path.exists(dest_folder):
            if int(input("資料夾已存在，請選擇保留請輸入 0 或刪除重新複製請輸入 1:")):
                print("您選擇1，重新複製資料夾")
                shutil.rmtree(dest_folder)
                shutil.copytree(source_folder, dest_folder)
            else:
                print("您選擇保留原本資料夾按 0，不複製")
        else:
            shutil.copytree(source_folder, dest_folder)
    else:
        print("查「無」指定資料夾")
    print("move_docutment(source_folder, dest_folder), done!")
    print("dest_folder:",dest_folder)
    print("-------------------------------------------------------------")   
    return dest_folder

PlanNo, DocNo = Get_DocNo()
print(PlanNo)
print(DocNo)
source_folder, dest_folder =Gen_path(PlanNo, DocNo)
print(source_folder)
print(dest_folder)
dest_folder = move_docutment(source_folder, dest_folder)
input("Press enter to exit...")
# PathWay : 1 = 雲端硬碟路徑  # 2 = 家庭硬碟路徑    # 3 = 公司硬碟路徑  # 0 = 指定路徑
# PlanNo :  1 = 興達計畫, 2 = 台中計畫, 99 = 指定路徑

## Read_EXCEL

# Read TC

In [ ]:
import os
import openpyxl
import pandas as pd

#輸入 目錄路徑
#輸出 1.螢幕相關資料   2.整個資料夾的Excel檔案彙整
def Read_TC_Excel(folder_path):
    #遍歷所有資料夾、檔案
    Excute_Num_block = []
    DrawingNo_block = []#J2
    Drawing_Title_block = []#F2
    DrawinVision_block = []#G2
    Letter_Num_block = []#A2
    Letter_Date_block = []#C2
    Letter_title_block =[]#I2
    file_path_block = []
    filetype = ".xlsm" if "CTC" in folder_path else ".xlsm"
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            #比對副檔名條件
            if file.endswith(filetype):
                file_path = os.path.join(root, file)
                #確認 1.無檔案 2.非有效Excel檔
                try:
                    # 開啟 Excel 檔案
                    workbook = openpyxl.load_workbook(file_path)
                    # 選取Data1工作表
                    worksheet = workbook['Data1']
                    #讀取表格文件數量
                    DrawingsNums = 0
                    for i in range(0,19):
                      if worksheet["A"+str(2+i)].value:
                        DrawingsNums = 1 + DrawingsNums
                      else:
                          break
                    #讀取資料
                    for i in range(0,DrawingsNums):
                        #print("讀取路徑", file_path, "第",i+1,"個檔案", )
                        DrawingNo_value = worksheet["J"+str(2+i)].value
                        Drawing_Title_value = worksheet["F"+str(2+i)].value
                        DrawinVision_value = worksheet["G"+str(2+i)].value
                        Letter_Num_value = worksheet["A2"].value
                        Letter_Date_value = worksheet["C2"].value
                        Letter_titl_value = worksheet["I2"].value
                        
                        Excute_Num_block.append(i)
                        DrawingNo_block.append(DrawingNo_value)
                        Drawing_Title_block.append(Drawing_Title_value)
                        DrawinVision_block.append(DrawinVision_value)
                        Letter_Num_block.append(Letter_Num_value)#來文號碼B2
                        Letter_Date_block.append(Letter_Date_value)#來文日期H2
                        Letter_title_block.append(Letter_titl_value)#來文標題O2
                        file_path_block.append(file_path)

                        print("------------------------")
                        print("批次序號",i)
                        print("圖號:",DrawingNo_value)
                        print("圖名:",Drawing_Title_value)
                        print("版次:",DrawinVision_value)
                        print("來文號碼:",worksheet["A2"].value)
                        print("來文日期:",worksheet["C2"].value)
                        print("來文名稱:",worksheet["I2"].value)
                        print("路徑",file_path)
                        print("------------------------")
                    workbook.close()
                except FileNotFoundError:
                    # 處理找不到檔案的錯誤
                    print("找不到檔案:", file_path)
                except openpyxl.utils.exceptions.InvalidFileException:
                    # 處理無效的 Excel 檔案的錯誤
                    print("無效的 Excel 檔案:", file_path)

    data = {'批次序號': Excute_Num_block,
            '圖號': DrawingNo_block,
            '圖名': Drawing_Title_block,
            '版次': DrawinVision_block,
            '來文號碼': Letter_Num_block,
            '來文日期': Letter_Date_block,
            '來文名稱': Letter_title_block,
            '路徑': file_path_block
            }
    df = pd.DataFrame(data)

    #輸出Excel檔案
    OutputName = 'TC_output.xlsx'
    Output_path = os.path.join(folder_path, OutputName)
    df.to_excel(Output_path, index=False)
    print("完成輸出檔案\n路徑:",Output_path,"\n檔名:",OutputName)

#全路徑參考目錄
Cloud_path = r"/content/sample_data/TC"
Home_path = r"C:\Users\OXO\OneDrive\01 Book\00 Test program\TC"
Company_path = r"D:\\00 興達計劃\\05 EPC提供資料\\TC/0TPC-TC(C0)-CD-23-0004"
path_map = {"0":Test_Path, "1" : Cloud_path, "2" : Home_path, "3" : Company_path}
#調試路徑
Test_Path = r"/content/sample_data/"

#qwe = input("請輸入路徑(因程式尚未完成，暫時性方案):\n測試路徑 0\n雲端路徑 1\n家庭路徑 2\n公司路徑3\n ")
#The_Excel_folder = path_map[qwe]
Test_Path = input("請輸入路徑:")
Read_TC_Excel(Test_Path)


# Convert .xlsb to .xlsx (only for NewHT Doc)

In [ ]:
pip install pyxlsb

In [ ]:
#解析xlsb成.xlsx
import pandas as pd
import datetime
now = datetime.datetime.now()
num = input("請輸入HT文號末4碼:")
file_name = "HT-D1-CTC-GEL-23-" + num + ".xlsb"
print("文件號碼:"+file_name)
The_Path = r'/content/sample_data/'
All_path = The_Path + file_name
print("處理路徑:"+All_path)
outpath_path = r'/content/sample_data/' + "HT-D1-CTC-GEL-23-" + num + ".xlsx"
print("輸出路徑:\n"+'\"' + outpath_path + '\"')

df = pd.read_excel(All_path, sheet_name='Data1', engine='pyxlsb')
df.to_excel(outpath_path)

請輸入HT文號末4碼:1796
文件號碼:HT-D1-CTC-GEL-23-1796.xlsb
處理路徑:/content/sample_data/HT-D1-CTC-GEL-23-1796.xlsb
輸出路徑:
"/content/sample_data/HT-D1-CTC-GEL-23-1796.xlsx"


In [ ]:
#自動全部掃描，資料夾，不須輸入關鍵字
import pandas as pd
import datetime
path = r'/content/sample_data/TC_TPC-'
num = input("請輸入HT文號末4碼:")
file_name = "HT-D1-CTC-GEL-23-" + num + ".xlsb"
print("文件號碼:"+file_name)
The_Path = r'/content/sample_data/'
All_path = The_Path + file_name
print("處理路徑:"+All_path)
outpath_path = r'/content/sample_data/' + "HT-D1-CTC-GEL-23-" + num + ".xlsx"
print("輸出路徑:\n"+'\"' + outpath_path + '\"')

df = pd.read_excel(All_path, sheet_name='Data1', engine='pyxlsb')
df.to_excel(outpath_path)

# Read HT

In [ ]:
#HT測試
# 指定要歷遍讀取的資料夾路徑●●●●●●●●●●●
import os
import openpyxl
import pandas as pd
#輸入 目錄路徑
#輸出 1.螢幕相關資料   2.整個資料夾的Excel檔案彙整

def Read_HT_Excel(folder_path):
    #遍歷所有資料夾、檔案
    Excute_Num_block = []
    DrawingNo_block = []#J2
    Drawing_Title_block = []#F2
    DrawinVision_block = []#G2
    Letter_Num_block = []#A2
    Letter_Date_block = []#C2
    Letter_title_block =[]#I2
    file_path_block = []
    filetype = ".xlsx"
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            #比對副檔名條件
            if file.endswith(filetype) * file.count("-") > 3:
                file_path = os.path.join(root, file)
                #確認 1.無檔案 2.非有效Excel檔
                try:
                    # 開啟 Excel 檔案
                    workbook = openpyxl.load_workbook(file_path)
                    # 選取Data1工作表
                    #●●注意因為.xlsb轉檔為.xlsx，所以已不須讀取分頁●●
                    #worksheet = workbook['Data1']
                    worksheet = workbook.active
                    #讀取表格文件數量
                    DrawingsNums = 0
                    for i in range(0,19):
                      if worksheet["B"+str(2+i)].value:
                        DrawingsNums = 1 + DrawingsNums
                      else:
                          break
                    print("文件數量",DrawingsNums)

                    #讀取資料
                    for i in range(0,DrawingsNums):
                        #print("讀取路徑", file_path, "第",i+1,"個檔案", )
                        #●●注意因為.xlsb轉檔為.xlsx，所以位置A>B, B>C●●
                        DrawingNo_value = worksheet["E"+str(2+i)].value#E2
                        Drawing_Title_value = worksheet["F"+str(2+i)].value
                        DrawinVision_value = worksheet["G"+str(2+i)].value
                        Letter_Num_value = worksheet["B2"].value
                        Letter_Date_value = worksheet["H2"].value
                        Letter_titl_value = worksheet["O2"].value
                        
                        Excute_Num_block.append(i)
                        DrawingNo_block.append(DrawingNo_value)
                        Drawing_Title_block.append(Drawing_Title_value)#F2
                        DrawinVision_block.append(DrawinVision_value)#F2
                        Letter_Num_block.append(Letter_Num_value)#來文號碼B2
                        Letter_Date_block.append(Letter_Date_value)#來文日期H2
                        Letter_title_block.append(Letter_titl_value)#來文標題O2
                        file_path_block.append(file_path)

                        print("------------------------")
                        print("批次序號",i)
                        print("圖號:",DrawingNo_value)
                        print("圖名:",Drawing_Title_value)
                        print("版次:",DrawinVision_value)
                        print("來文號碼:",worksheet["B2"].value)
                        print("來文日期:",worksheet["H2"].value)
                        print("來文名稱:",worksheet["O2"].value)
                        print("路徑",file_path)
                        print("------------------------")
                        print(Letter_titl_value, DrawinVision_value, Letter_Num_value, Letter_Date_value)
                        return Letter_titl_value, DrawinVision_value, Letter_Num_value, Letter_Date_value
                    workbook.close()
                except FileNotFoundError:
                    # 處理找不到檔案的錯誤
                    print("找不到檔案:", file_path)
                except openpyxl.utils.exceptions.InvalidFileException:
                    # 處理無效的 Excel 檔案的錯誤
                    print("無效的 Excel 檔案:", file_path)

    data = {'批次序號': Excute_Num_block,
            '圖號': DrawingNo_block,
            '圖名': Drawing_Title_block,
            '版次': DrawinVision_block,
            '來文號碼': Letter_Num_block,
            '來文日期': Letter_Date_block,
            '來文名稱': Letter_title_block,
            '路徑': file_path_block
            }
    df = pd.DataFrame(data)

    #輸出Excel檔案
    OutputName = 'HT_output.xlsx'
    Output_path = os.path.join(folder_path, OutputName)
    df.to_excel(Output_path, index=False)
    print("完成輸出檔案\n路徑:",Output_path,"\n檔名:",OutputName)


#全路徑參考目錄
Test_Path = r"/content/sample_data/"
Cloud_path = r"/content/sample_data/TC"
Home_path = r"C:\Users\OXO\OneDrive\01 Book\00 Test program\TC"
Company_path = r"D:\\00 興達計劃\\05 EPC提供資料\\TC/0TPC-TC(C0)-CD-23-0004"
path_map = {"0":Test_Path, "1" : Cloud_path, "2" : Home_path, "3" : Company_path}
#調試路徑

Read_HT_Excel(Test_Path)

完成輸出檔案
路徑: /content/sample_data/HT_output.xlsx 
檔名: HT_output.xlsx


#TC

In [ ]:
# 指定要歷遍讀取的資料夾路徑●●●●●●●●●●●
import os
import openpyxl
import pandas as pd

#輸入 目錄路徑
#輸出 1.螢幕相關資料   2.整個資料夾的Excel檔案彙整
def Read_TC_Excel(folder_path):
    #遍歷所有資料夾、檔案
    Excute_Num_block = []
    DrawingNo_block = []#J2
    Drawing_Title_block = []#F2
    DrawinVision_block = []#G2
    Letter_Num_block = []#A2
    Letter_Date_block = []#C2
    Letter_title_block =[]#I2
    file_path_block = []
    filetype = ".xlsm" if "CTC" in folder_path else ".xlsm"
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            #比對副檔名條件
            if file.endswith(filetype):
                file_path = os.path.join(root, file)
                #確認 1.無檔案 2.非有效Excel檔
                try:
                    # 開啟 Excel 檔案
                    workbook = openpyxl.load_workbook(file_path)
                    # 選取Data1工作表
                    worksheet = workbook['Data1']
                    #讀取表格文件數量
                    DrawingsNums = 0
                    for i in range(0,19):
                      if worksheet["A"+str(2+i)].value:
                        DrawingsNums = 1 + DrawingsNums
                      else:
                          break
                    #讀取資料
                    for i in range(0,DrawingsNums):
                        #print("讀取路徑", file_path, "第",i+1,"個檔案", )
                        DrawingNo_value = worksheet["J"+str(2+i)].value
                        Drawing_Title_value = worksheet["F"+str(2+i)].value
                        DrawinVision_value = worksheet["G"+str(2+i)].value
                        Letter_Num_value = worksheet["A2"].value
                        Letter_Date_value = worksheet["D2"].value
                        Letter_titl_value = worksheet["I2"].value
                        
                        Excute_Num_block.append(i)
                        DrawingNo_block.append(DrawingNo_value)
                        Drawing_Title_block.append(Drawing_Title_value)
                        DrawinVision_block.append(DrawinVision_value)
                        Letter_Num_block.append(Letter_Num_value)#來文號碼B2
                        Letter_Date_block.append(Letter_Date_value)#來文日期H2
                        Letter_title_block.append(Letter_titl_value)#來文標題O2
                        file_path_block.append(file_path)

                        print("------------------------")
                        print("批次序號",i)
                        print("圖號:",DrawingNo_value)
                        print("圖名:",Drawing_Title_value)
                        print("版次:",DrawinVision_value)
                        print("來文號碼:",Letter_Num_value)
                        print("來文日期:",Letter_Date_value)
                        print("來文名稱:",Letter_titl_value)
                        print("路徑",file_path)
                        print("------------------------")
                        print(Letter_titl_value, DrawinVision_value, Letter_Num_value, Letter_Date_value)
                        return Letter_titl_value, DrawinVision_value, Letter_Num_value, Letter_Date_value
                    workbook.close()
                except FileNotFoundError:
                    # 處理找不到檔案的錯誤
                    print("找不到檔案:", file_path)
                except openpyxl.utils.exceptions.InvalidFileException:
                    # 處理無效的 Excel 檔案的錯誤
                    print("無效的 Excel 檔案:", file_path)

    data = {'批次序號': Excute_Num_block,
            '圖號': DrawingNo_block,
            '圖名': Drawing_Title_block,
            '版次': DrawinVision_block,
            '來文號碼': Letter_Num_block,
            '來文日期': Letter_Date_block,
            '來文名稱': Letter_title_block,
            '路徑': file_path_block
            }
    df = pd.DataFrame(data)

    #輸出Excel檔案
    OutputName = 'TC_output.xlsx'
    Output_path = os.path.join(folder_path, OutputName)
    df.to_excel(Output_path, index=False)
    print("完成輸出檔案\n路徑:",Output_path,"\n檔名:",OutputName)


#HT

In [ ]:
def Read_HT_Excel(folder_path):
    #遍歷所有資料夾、檔案
    Excute_Num_block = []
    DrawingNo_block = []#J2
    Drawing_Title_block = []#F2
    DrawinVision_block = []#G2
    Letter_Num_block = []#A2
    Letter_Date_block = []#C2
    Letter_title_block =[]#I2
    file_path_block = []
    filetype = ".xlsx"
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            #比對副檔名條件
            if file.endswith(filetype) * file.count("-") > 3:
                file_path = os.path.join(root, file)
                #確認 1.無檔案 2.非有效Excel檔
                try:
                    # 開啟 Excel 檔案
                    workbook = openpyxl.load_workbook(file_path)
                    # 選取Data1工作表
                    #●●注意因為.xlsb轉檔為.xlsx，所以已不須讀取分頁●●
                    #worksheet = workbook['Data1']
                    worksheet = workbook.active
                    #讀取表格文件數量
                    DrawingsNums = 0
                    for i in range(0,19):
                      if worksheet["B"+str(2+i)].value:
                        DrawingsNums = 1 + DrawingsNums
                      else:
                          break
                    print("文件數量",DrawingsNums)

                    #讀取資料
                    for i in range(0,DrawingsNums):
                        #print("讀取路徑", file_path, "第",i+1,"個檔案", )
                        #●●注意因為.xlsb轉檔為.xlsx，所以位置A>B, B>C●●
                        DrawingNo_value = worksheet["E"+str(2+i)].value#E2
                        Drawing_Title_value = worksheet["F"+str(2+i)].value
                        DrawinVision_value = worksheet["G"+str(2+i)].value
                        Letter_Num_value = worksheet["B2"].value
                        Letter_Date_value = worksheet["H2"].value
                        Letter_titl_value = worksheet["O2"].value
                        
                        Excute_Num_block.append(i)
                        DrawingNo_block.append(DrawingNo_value)
                        Drawing_Title_block.append(Drawing_Title_value)#F2
                        DrawinVision_block.append(DrawinVision_value)#F2
                        Letter_Num_block.append(Letter_Num_value)#來文號碼B2
                        Letter_Date_block.append(Letter_Date_value)#來文日期H2
                        Letter_title_block.append(Letter_titl_value)#來文標題O2
                        file_path_block.append(file_path)

                        print("------------------------")
                        print("批次序號",i)
                        print("圖號:",DrawingNo_value)
                        print("圖名:",Drawing_Title_value)
                        print("版次:",DrawinVision_value)
                        print("來文號碼:", Letter_Num_value)
                        print("來文日期:", Letter_Date_value)
                        print("來文名稱:", Letter_titl_value)
                        print("路徑",file_path)
                        print("------------------------")
                        print(x, y, z, zz)
                        return Letter_titl_value, DrawinVision_value, Letter_Num_value, Letter_Date_value
                    workbook.close()
                except FileNotFoundError:
                    # 處理找不到檔案的錯誤
                    print("找不到檔案:", file_path)
                except openpyxl.utils.exceptions.InvalidFileException:
                    # 處理無效的 Excel 檔案的錯誤
                    print("無效的 Excel 檔案:", file_path)

    data = {'批次序號': Excute_Num_block,
            '圖號': DrawingNo_block,
            '圖名': Drawing_Title_block,
            '版次': DrawinVision_block,
            '來文號碼': Letter_Num_block,
            '來文日期': Letter_Date_block,
            '來文名稱': Letter_title_block,
            '路徑': file_path_block
            }
    df = pd.DataFrame(data)

    #輸出Excel檔案
    OutputName = 'HT_output.xlsx'
    Output_path = os.path.join(folder_path, OutputName)
    df.to_excel(Output_path, index=False)
    print("完成輸出檔案\n路徑:",Output_path,"\n檔名:",OutputName)



#Excute

In [ ]:
#全路徑參考目錄
Test_Path = r"/content/sample_data/"
Cloud_path = r"/content/sample_data/TC"
Home_path = r"C:\Users\OXO\OneDrive\01 Book\00 Test program\TC"
Company_path = r"D:\\00 興達計劃\\05 EPC提供資料\\TC/0TPC-TC(C0)-CD-23-0004"
path_map = {"0":Test_Path, "1" : Cloud_path, "2" : Home_path, "3" : Company_path}
#調試路徑


#qwe = input("請輸入路徑(因程式尚未完成，暫時性方案):\n測試路徑 0\n雲端路徑 1\n家庭路徑 2\n公司路徑3\n ")
#The_Excel_folder = path_map[qwe]
result0 = Read_TC_Excel(Test_Path)
result1 = Read_HT_Excel(Test_Path)
print(result0)

------------------------
批次序號 0
圖號: TC0-1-UMM02-T9411
圖名: Motor Shop test and inspection plan / procedure for Self priming Pump (Recovery Water Treatment System / Wastewater Collection System)
版次: C
來文號碼: TPC-TC(C0)-CD-23-1700
來文日期: 2023/05/23
來文名稱: Motor Shop test and inspection plan / procedure for Self priming Pump (Recovery Water Treatment System / Wastewater Collection System)
路徑 /content/sample_data/TPC-TC(C0)-CD-23-1700.xlsm
------------------------
Motor Shop test and inspection plan / procedure for Self priming Pump (Recovery Water Treatment System / Wastewater Collection System) C TPC-TC(C0)-CD-23-1700 2023/05/23
完成輸出檔案
路徑: /content/sample_data/HT_output.xlsx 
檔名: HT_output.xlsx
('Motor Shop test and inspection plan / procedure for Self priming Pump (Recovery Water Treatment System / Wastewater Collection System)', 'C', 'TPC-TC(C0)-CD-23-1700', '2023/05/23')


#合併輸出檔案，自動分析是否重複

#文字輸出，生成文稿

In [ ]:
import datetime#處理年月用

#-----------------在此輸入ReadHT結果------------------"
Doc_content = result0[0]
Doc_Vision = result0[1]
Doc_No = result0[2]
Doc_Date = result0[3]

#-----------------在此輸入相關對象結果------------------"
My_company_num = Consult_Company_num = 2
My_company = ["", "南部施工處", "中部施工處", "興達發電廠", "台中發電廠"]
Consult_Company = ["", "吉興公司", "泰興公司", "GE/CTCI"]
Pages = 3

date_obj = datetime.datetime.strptime(Doc_Date, "%Y/%m/%d")
month = date_obj.strftime("%m")
day = date_obj.strftime("%d")

Plan_Name = "興達" if "CTC" in Doc_No else "台中"
contents0 = "本文係" + My_company[My_company_num] + "對統包商提送「" + Doc_content + "」" + "Rev." + Doc_Vision +"所提審查意見(共" + str(Pages) + "頁)" + "，未逾合約規範，已電傳" + Consult_Company[Consult_Company_num] + "，擬陳閱後文存。"
contents1 = "檢送" + Plan_Name + "電廠燃氣機組更新改建計畫" + Doc_content + "Rev." + Doc_Vision + "，" + My_company[My_company_num] + "之審查意見（如附，共" + str(Pages) + "頁）供卓參，請查照。"
contents2 = "依據GE/CTCI 112年" + month + "月" + day +"日" + Doc_No + "號辦理。"
contents4 = "本文係統包商提送「" + Doc_content + "」" + "Rev." + Doc_Vision +"，本組無意見，已Email通知" + Consult_Company[Consult_Company_num] + "公司" + "，擬陳閱後文存。"
print("----------------他單位審查意見簽辦------------------")
print(contents0)
print("----------------傳真------------------")
print(contents1)
print(contents2)
print("----------------主辦簽辦------------------")
print(contents4)

----------------他單位審查意見簽辦------------------
本文係中部施工處對統包商提送「Motor Shop test and inspection plan / procedure for Self priming Pump (Recovery Water Treatment System / Wastewater Collection System)」Rev.C所提審查意見(共3頁)，未逾合約規範，已電傳泰興公司，擬陳閱後文存。
----------------傳真------------------
檢送台中電廠燃氣機組更新改建計畫Motor Shop test and inspection plan / procedure for Self priming Pump (Recovery Water Treatment System / Wastewater Collection System)Rev.C，中部施工處之審查意見（如附，共3頁）供卓參，請查照。
依據GE/CTCI 112年05月23日TPC-TC(C0)-CD-23-1700號辦理。
----------------主辦簽辦------------------
本文係統包商提送「Motor Shop test and inspection plan / procedure for Self priming Pump (Recovery Water Treatment System / Wastewater Collection System)」Rev.C，本組無意見，已Email通知泰興公司公司，擬陳閱後文存。


# 封裝

In [ ]:
pip install pyinstaller
cd C:\Users\OXO\OneDrive\01 Book\00 Test program
pyinstaller -F Pack.py

#Test

In [ ]:
DrawingNo_block = []
t = [1,2]
DrawingNo_value = t[1]#E2
DrawingNo_block.append(DrawingNo_value)
print(DrawingNo_block)

[2]


合併新檔、舊檔


In [ ]:
import pandas as pd

# 將 Excel 檔案讀取為 Pandas DataFrame
df_A = pd.read_excel("/content/sample_data/1.xlsx")
df_B = pd.read_excel("/content/sample_data/2.xlsx")

# 合併 DataFrame, 並使用 drop_duplicate() 刪除重複的行
df_merged = pd.concat([df_A, df_B]).drop_duplicates()

# 將合併後的 DataFrame 寫入新的 Excel 檔案
df_merged.to_excel("/content/sample_data/merged.xlsx", index=False)
